In [5]:
import os
import re
import torch
import json
import nltk
import unicodedata
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from datasets import Dataset
from typing import List, Tuple
from transformers import AutoTokenizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)

Cuda and stuff

In [6]:
# Check whether CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.rand(3, 3).to(device)
print(f'Tensor is on: {x.device}')

Tensor is on: cpu


In [7]:
# print cuda version
print(torch.version.cuda)

None


In [8]:
# check the current working directory
os.chdir('C:/Users/lamem/OneDrive/Documents/GHD/Microsoft-Learn-Location-Mention-Recognition-Challenge')
#os.chdir('C:/Users/abelm/OneDrive/Documents/GitHub/Microsoft-Learn-Location-Mention-Recognition-Challenge')
print(os.getcwd())

C:\Users\lamem\OneDrive\Documents\GHD\Microsoft-Learn-Location-Mention-Recognition-Challenge


In [9]:
new = pd.read_csv("lewa/Train_1.csv")
new.head()

,tweet_id,text,location
0,ID_1001136212718088192,NaN,EllicottCity
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday,...",Maryland
2,ID_1001136950345109504,State of emergency declared for Maryland flood...,Maryland
3,ID_1001137334056833024,Other parts of Maryland also saw significant d...,Baltimore Maryland
4,ID_1001138374923579392,"Catastrophic Flooding Slams Ellicott City, Mar...",Ellicott City Maryland


In [10]:
# initialize stopwords
stop_words = set(stopwords.words('english'))

# text clean helper function
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # remove mentions only
    text = re.sub(r'@\w+', '', text)
    
    # Remove special characters, numbers, and punctuations, keeping spaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove stopwords
    words = text.split()  # Split by spaces without tokenizing
    cleaned_words = [word for word in words if word not in stop_words]
    
    # Join words back into a single string
    return ' '.join(cleaned_words)

In [11]:
# Load your  
df_t = new.copy() 
df_t = df_t.dropna(subset = "text")
# clean the text
df_t['text'] = df_t['text'].apply(clean_text)
df_t.head()

,tweet_id,text,location
1,ID_1001136696589631488,Flash floods struck Maryland city Sunday washi...,Maryland
2,ID_1001136950345109504,State emergency declared Maryland flooding via,Maryland
3,ID_1001137334056833024,Other parts Maryland also saw significant dama...,Baltimore Maryland
4,ID_1001138374923579392,Catastrophic Flooding Slams Ellicott City Mary...,Ellicott City Maryland
5,ID_1001138377717157888,WATCH missing flash FLOODING devastates Ellico...,Ellicott City Maryland


In [12]:
df_t.shape

(16448, 3)

In [13]:
# separate data into train and dev
trainData, devData = train_test_split(df_t, test_size=0.15, random_state=42)

# save data to a csv tainData
trainData.to_csv('lewa/Train-dropna.csv', index=False)

# save data to a csv devData
devData.to_csv('lewa/Dev-dropna.csv', index=False)

In [14]:
import csv
import json

def csv_to_json(csv_file_path, json_file_path):
    # Open the CSV file and create a list of dictionaries
    with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        data = [row for row in csv_reader]

    # Write the list of dictionaries to a JSON file
    with open(json_file_path, mode='w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4)

# Example usage
#csv_file_path = 'testData/Train-dropna.csv'  # Path to your CSV file
#json_file_path = 'testData/output_file.json'  # Path where the JSON will be saved
csv_to_json('lewa/Train-dropna.csv', 'lewa/train_file.json')
csv_to_json('lewa/Dev-dropna.csv', 'lewa/dev_file.json')


IDENTIFY location from sentences 

In [15]:
import spacy
import string

# Load spaCy's English model
nlp = spacy.load("en_core_web_lg")

def remove_hashtag(text):
    # Find hashtags and process each one
    return re.sub(r'#(\w+)', lambda m: ' '.join(re.findall(r'[A-Z][^A-Z]*|[a-z]+', m.group(1))), text)

# Function to clean and preprocess text (tokenization, normalization, lemmatization)
def preprocess_text(text):
    # Remove special characters (keeping alphanumeric characters and spaces)
    text = remove_hashtag(text)  # Remove hashtags and separate words
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)

    doc = nlp(text)
    cleaned_tokens = []
    for token in doc:
        # Normalize: Convert to lowercase and remove punctuation
        if token.text not in string.punctuation:
            cleaned_tokens.append(token.lemma_.lower())  # Lemmatize and lowercase
    return " ".join(cleaned_tokens)

# Function to extract locations as entities after preprocessing
def extract_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        if ent.label_ == 'GPE':  # 'GPE' is for locations
            entities.append({
                'entity': ent.text,
                'label': ent.label_,
                'start_char': ent.start_char,
                'end_char': ent.end_char
            })
    return entities

# Function to process a JSON file
def process_json_file(file_path):
    # Load JSON data
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Convert the JSON data to a DataFrame
    df = pd.DataFrame(data)

    # Preprocess text and extract entities
    df['cleaned_text'] = df['text'].apply(preprocess_text)
    df['entities'] = df['cleaned_text'].apply(extract_entities)

    return df

# Paths to your two JSON files
file_path_1 = 'lewa/train_file.json'
file_path_2 = 'lewa/dev_file.json'

# Process both files
df1 = process_json_file(file_path_1)
df2 = process_json_file(file_path_2)


# If you want to save them separately, you can use the individual DataFrames:
# Save the updated data for file1
with open('lewa/train_entities.json', 'w') as f:
    json.dump(df1.to_dict(orient='records'), f, indent=4)

# Save the updated data for file2
with open('lewa/dev_entities.json', 'w') as f:
    json.dump(df2.to_dict(orient='records'), f, indent=4)

 Merge the extracted_locations with the Existing location Column

In [16]:
# helper function to merge location and entities, taking file paths for input and output
def merge_entities(input_file_path, output_file_path):
    # Read the unmerged JSON file
    with open(input_file_path, 'r') as infile:
        data = json.load(infile)
    
    for entry in data:
        location = entry.get("location")
        entities = entry.get("entities", [])
        
        # Create the merged_entities field with only entities present in the entities list
        entry["merged_entities"] = [ent["entity"] for ent in entities]

        merged_entities = [ent["entity"] for ent in entities]

        # If location is present and valid, merge it with entities
        if location:
            merged_entities.append(location)

        # Update the entry with merged entities
        entry["merged_entities"] = merged_entities
    
    # Write the updated data to the output JSON file
    with open(output_file_path, 'w') as outfile:
        json.dump(data, outfile, indent=4)


In [17]:
# Paths to your two JSON files
input_file_path = 'lewa/train_entities.json'
output_file_path = 'lewa/train_merged_entities.json'

# Merge entities for the first file
merge_entities(input_file_path, output_file_path)

# Paths to your two JSON files
input_file_path = 'lewa/dev_entities.json'
output_file_path = 'lewa/dev_merged_entities.json'

# Merge entities for the first file
merge_entities(input_file_path, output_file_path)

BIOES TAGGING

In [18]:
# Helper function to read JSON file and return a DataFrame with specific columns
def json_to_csv(input_file_path, output_csv_path):
    # Read the JSON file
    with open(input_file_path, 'r') as infile:
        data = json.load(infile)
    
    # Create a list to hold the data for the DataFrame
    rows = []
    
    for entry in data:
        tweet_id = entry.get("tweet_id")
        cleaned_text = entry.get("cleaned_text")
        merged_entities = entry.get("merged_entities", [])
        
        # Convert the list of entities into a single string (join them with a comma and space)
        location = ' '.join(merged_entities) if merged_entities else None
        
        # Add the row to the list
        rows.append({
            "tweet_id": tweet_id,
            "text": cleaned_text,  # Rename cleaned_text to text
            "location": location  # Convert merged_entities to a plain text string
        })
    
    # Create a DataFrame from the list of rows
    df = pd.DataFrame(rows)
    
    # Save the DataFrame to a CSV file
    df.to_csv(output_csv_path, index=False)

In [19]:
# save merged_train_entities to a csv
train_file_path = 'lewa/train_merged_entities.json'
train_csv_path = 'lewa/train_merged_entities.csv'
json_to_csv(train_file_path, train_csv_path)

# save merged_dev_entities to a csv
dev_file_path = 'lewa/dev_merged_entities.json' 
dev_csv_path = 'lewa/dev_merged_entities.csv'
json_to_csv(dev_file_path, dev_csv_path)

tagging

In [20]:
def bioes_tagging(text, merged_entities):
    """
    Apply BIOES tagging to a tokenized text based on merged entity spans.
    
    :param text: List of tokenized words.
    :param merged_entities: List of merged location entities in the sentence (already a list).
    :return: List of tuples (word, tag) in BIOES format.
    """
    tags = ['O'] * len(text)
    
    # Process each merged entity (location)
    for entity in merged_entities:
        entity_tokens = entity.split()  # Assuming each entity is a string of tokens
        entity_length = len(entity_tokens)

        # Apply BIOES tags for each token in the entity
        for i, token in enumerate(entity_tokens):
            try:
                token_index = text.index(token)
            except ValueError:
                continue  # Skip tokens not found in the sentence

            if entity_length == 1:
                tags[token_index] = 'S-LOCATION'
            elif i == 0:
                tags[token_index] = 'B-LOCATION'
            elif i == entity_length - 1:
                tags[token_index] = 'E-LOCATION'
            else:
                tags[token_index] = 'I-LOCATION'
    
    return list(zip(text, tags))

def process_json(input_file, output_file):
    # Read the JSON file
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    output_data = []

    # Process each entry in the JSON file
    for entry in data:
        text = entry['text']  # Assuming 'text' field has the sentence
        merged_entities = entry.get('merged_entities', [])  # No split needed, it's already a list
        
        # Tokenize the text (basic tokenization)
        tokens = text.split()

        # Apply BIOES tagging
        tagged_tokens = bioes_tagging(tokens, merged_entities)
        
        # Prepare output format
        for token, tag in tagged_tokens:
            output_data.append(f"{token}\t{tag}")
        output_data.append("")  # New line after each sentence

    # Write the output to a text file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("\n".join(output_data))

In [21]:
# Example usage
process_json('lewa/train_merged_entities.json', 'lewa/train_bioes.txt')
process_json('lewa/dev_merged_entities.json', 'lewa/dev_bioes.txt')